In [ ]:
#basic stuff :
!pip install langchain langchain-openai langchain-community openai python-dotenv


INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32

In [ ]:
from google.colab import userdata
import os

# Fetch your stored Colab secrets
NEBIUS_KEY = userdata.get("NEBIUS_API_KEY")
OPENAI_KEY = userdata.get("OPENAI_API_KEY")

# Check what loaded (safe because values are hidden)
print("NEBIUS_KEY found:", bool(NEBIUS_KEY))
print("OPENAI_KEY found:", bool(OPENAI_KEY))


NEBIUS_KEY found: True
OPENAI_KEY found: True


In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

load_dotenv()
# Assign them to env vars so LangChain’s validation passes
os.environ["NEBIUS_API_KEY"] = NEBIUS_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_KEY or NEBIUS_KEY  # LangChain needs this

NEBIUS_BASE = "https://api.studio.nebius.com/v1"

llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0.6,
    max_tokens=512,
    openai_api_base=NEBIUS_BASE,
    openai_api_key=NEBIUS_KEY
)


/tmp/ipython-input-547857652.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


##Building a simple agent using wikepedia

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=ffa7e3d1604d3af1956ae0e24aee96ba9fa8d1968eb9da8bcf981ce9650bede7
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


##Single agent with one tool

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, load_tools

tools = load_tools(["wikipedia"], llm=llm)


agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Who is the president of India and what is 345 * 23?")


/tmp/ipython-input-1933466031.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipython-input-1933466031.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who is the president of India and what is 345 * 23?")




> Entering new AgentExecutor chain...
Thought: I need to find the current president of India and perform a mathematical calculation.
Action: wikipedia
Action Input: president of India
Observation: Page: President of India
Summary: The president of India  (ISO: Bhārata kē Rāṣṭrapati) is the head of state of the Republic of India. The president is the nominal head of the executive, the first citizen of the country, and the supreme commander of the Indian Armed Forces. Droupadi Murmu is the 15th and current president, having taken office on 25 July 2022.
The office of president was created when India's  constitution came into force and it became a republic on 26 January 1950. The president is indirectly elected by an electoral college comprising both houses of the Parliament of India and the legislative assemblies of each of India's states and territories, who themselves are all directly elected by the citizens.
The President ranks 1st in the Order of Precedence of India as per Article 

'The final answer is 7935.'

##Single agent with multiple tools

In [ ]:
from langchain.tools import Tool
import re, math

def safe_calc(expr: str):
    expr = "".join(re.findall(r"[0-9+\-*/(). ]+", expr))
    return str(eval(expr))


calc_tool = Tool(
    name="Calculator",
    func=safe_calc,
    description="Safely evaluate simple arithmetic expressions."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType, load_tools


tools = load_tools(["wikipedia", "llm-math"], llm=llm) + [calc_tool]

agent = initialize_agent(
    tools,
    llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Who is the president of India and what is 345 * 23?")




> Entering new AgentExecutor chain...
Thought: I need to find the current president of India and calculate the product of 345 and 23.
Action: wikipedia
Action Input: president of India
Observation: Page: President of India
Summary: The president of India  (ISO: Bhārata kē Rāṣṭrapati) is the head of state of the Republic of India. The president is the nominal head of the executive, the first citizen of the country, and the supreme commander of the Indian Armed Forces. Droupadi Murmu is the 15th and current president, having taken office on 25 July 2022.
The office of president was created when India's  constitution came into force and it became a republic on 26 January 1950. The president is indirectly elected by an electoral college comprising both houses of the Parliament of India and the legislative assemblies of each of India's states and territories, who themselves are all directly elected by the citizens.
The President ranks 1st in the Order of Precedence of India as per Article

'The final answer is $\\boxed{7935}$.'

The final answer is $\boxed{7935}$.


##getting both answers using prompting

In [ ]:
query = (
  "Find the current president of India AND calculate 345 * 23. "
  "When you finish: provide a single final response that includes BOTH items, "
  "in this exact format:\n"
  "President: <name>\nCalculation: <result>\n"
)
print(agent.run(query))




> Entering new AgentExecutor chain...
Thought: I need to find the current president of India and calculate the product of 345 and 23.
Action: wikipedia
Action Input: "current president of India"
Observation: Page: President of India
Summary: The president of India  (ISO: Bhārata kē Rāṣṭrapati) is the head of state of the Republic of India. The president is the nominal head of the executive, the first citizen of the country, and the supreme commander of the Indian Armed Forces. Droupadi Murmu is the 15th and current president, having taken office on 25 July 2022.
The office of president was created when India's  constitution came into force and it became a republic on 26 January 1950. The president is indirectly elected by an electoral college comprising both houses of the Parliament of India and the legislative assemblies of each of India's states and territories, who themselves are all directly elected by the citizens.
The President ranks 1st in the Order of Precedence of India as p